## Crafting Better Prompts for LLMs:

In this guide, we’ll explore practical ways to interact with Large Language Models (LLMs) and make your prompts smarter and more effective. Working with LLMs isn’t just about asking questions, it’s about asking them well. The right context and structure can transform a basic query into a highly accurate and relevant response. That’s exactly what we’ll focus on here: **how to enrich your prompts with additional information so the model delivers better, more precise answers**.

Here’s what we will do in this blog:

* **How to interact with an LLM effectively** - from sending single prompts to managing multi-turn conversations.
* **How to enhance your prompts with extra data and context** - so the responses you get are richer, more accurate, and truly useful.

### Setting up some functions to use frequently

In [1]:
import json
from typing import List, Dict

In [2]:
from langchain_ollama import ChatOllama
from langchain.schema import HumanMessage, SystemMessage, AIMessage

#### `generate_with_single_input` – Your Gateway to Simple LLM Calls

The `generate_with_single_input` function is designed to make it easy to generate text from a language model using just **one input prompt**. Think of it as your simplest entry point into interacting with an LLM, perfect for quick experiments or straightforward use cases.

For now, we’ll keep things simple and focus on the essential parameters that let you control the model’s behavior without overwhelming complexity.

Here’s what you can work with:

* **`prompt` (str)**: The actual text you want to send to the model. This is your question, instruction, or context or all of it together.
* **`role` (str)**: Defines the role of the message sender (e.g., `"user"`, `"system"`). Defaults to `"user"`.
* **`temperature` (float)**: Controls creativity. Lower values (e.g., `0.1`) keep responses focused and deterministic, while higher values make them more diverse.
* **`top_p` (float)**: A sampling method to limit randomness by focusing on top probable tokens.
* **`max_tokens` (int)**: The maximum length of the response you want from the model.
* **`model` (str)**: The LLM you want to use, such as I used `"llama3.1:8b"`from Ollama.
* **`frequency_penalty` (float)**: Reduces repeated words or phrases in the response, making the output less redundant. Default to 0.5
* **`presence_penalty` (float)**: Encourages the model to introduce new ideas and avoid sticking only to what’s already mentioned. Default to 0.3


This function keeps the process lightweight and straightforward. Just provide a prompt, tweak a few knobs like `temperature` and `top_p`, and you’re ready to see the model in action.

In [3]:
def generate_with_single_input(
                            prompt: str,
                            role: str = 'user',
                            top_p: float = 0.1,
                            temperature: float = 0.1,
                            max_tokens: int = 500,
                            model: str = "llama3.1:8b",
                            **kwargs
                                ):


    llm = ChatOllama(
        model = model,
        temperature = temperature,
        top_p= top_p,
        frequency_penalty = 0.5,
        presence_penalty = 0.3,
        max_tokens = max_tokens
        )

    
    role_map = {"user" : HumanMessage,
                "system" : SystemMessage,
                "assistant" : AIMessage}

    # OpenAI Style payload - usually the hyperparameters are placed here if calling for OpenAI API
    payload = {
        "model": model,
        "messages": [
            {"role": role, "content": prompt}
        ]
    }

    content_passed = payload['messages'][0]['content']
    role = role_map[payload['messages'][0]['role'].lower()]
    messages = [role(content=content_passed)] # Ollama do not accept input in dictionary format or OpenAI style format

    response = llm.invoke(messages)

    response_role = "assistant" if response.type == "ai" else response.type
    response_type = {'Role':response_role}

    # Convert to dictionary
    json_dict = response.model_dump()
    json_dict.update(response_type)  # Adding response type with the response dictionary

    return json_dict

#### **`generate_with_multiple_input` – For Multi-Turn Conversations**

The `generate_with_multiple_input` function is built to handle **multiple messages in a conversational flow**, making it ideal for scenarios where context from previous exchanges matters.

The input follows a simple structure:
* **`role`**: Defines who is speaking (`"user"`, `"system"`, or `"assistant"`).
* **`content`**: The actual text of the message.

**Parameters**

* **`messages` (List\[Dict])**: A list of message objects, each containing `role` and `content` to represent the conversation turns.
* **`max_tokens` (int)**: Sets the maximum token limit for the model’s response.

Other parameters are same as in first function.
This function gives you the flexibility to include **system instructions, user queries, and assistant responses**, ensuring the model stays aligned with the ongoing context.


In [4]:
def generate_with_multiple_input(
                            messages: List[Dict],
                            top_p: float = 0.1,
                            temperature: float = 0.1,
                            max_tokens: int = 500,
                            model: str = "llama3.1:8b",
                            **kwargs
                                ):


    llm = ChatOllama(
        model = model,
        temperature = temperature,
        top_p= top_p,
        frequency_penalty = 0.5,
        presence_penalty = 0.3,
        max_tokens = max_tokens
        )

    
    role_map = {"user" : HumanMessage,
                "system" : SystemMessage,
                "assistant" : AIMessage}


    converted_messages = [role_map[m["role"].lower()](content=m["content"]) for m in messages]

    response = llm.invoke(converted_messages)
    response_role = "assistant" if response.type == "ai" else response.type

    
    ## Start - This block is for returning the final result only and not the conversation log
    # Convert to dictionary
    #json_dict =  {
    #    "role": response_role,
    #    "content": response.content
    #}
    #return json_dict
    ## End

    # This is for returning the full conversation log passed
    conversation_log = [
        {"role": m["role"], "content": m["content"]} for m in messages
    ]
    conversation_log.append({"role": response_role, "content": response.content})

    return conversation_log

    

### Try out the functions!

In [5]:
output = generate_with_single_input(
    prompt = 'What is the capital of India?')

In [6]:
print('Role: ', output['Role'])
print('Content: ', output['content'])

Role:  assistant
Content:  The capital of India is New Delhi.


Great! Now with multiple turn inputs

In [7]:
messages = [
    {'role':'user', 'content': 'Where is India?'},
    {'role':'assistant', 'content':'India is in Asia'},
    {'role':'user', 'content':'Who is the Prime Minister?'}
]

output = generate_with_multiple_input(messages, max_tokens=100, temperature=1.5)

In [8]:
output

[{'role': 'user', 'content': 'Where is India?'},
 {'role': 'assistant', 'content': 'India is in Asia'},
 {'role': 'user', 'content': 'Who is the Prime Minister?'},
 {'role': 'assistant',
  'content': "As of my last update (May 2023), Narendra Modi has been serving as the Prime Minister of India since May 2014. However, please note that this information may change over time due to elections or other political developments.\n\nIf you're looking for more up-to-date information, I recommend checking a reliable news source or the official website of the Government of India for the latest updates on the current Prime Minister."}]

In [9]:
# This is used when a single output is returned and not conversation log as above
#print("Role:", output['role'])
#print("Content:", output['content'])

### Introducing Augmenting

Now we will learn how to incorporate data into a prompt before passing it to a LLM.
We will use a small dataset of JSON files containing information about houses. This example will help you understand how to augment prompts in the context of Retrieval-Augmented Generation (RAG).

The dataset is simple: a list where each element represents a house as a dictionary of attributes.

In [10]:
house_data = [
    {
        "address": "123 Maple Street",
        "city": "Springfield",
        "state": "IL",
        "zip": "62701",
        "bedrooms": 3,
        "bathrooms": 2,
        "square_feet": 1500,
        "price": 230000,
        "year_built": 1998
    },
    {
        "address": "456 Elm Avenue",
        "city": "Shelbyville",
        "state": "TN",
        "zip": "37160",
        "bedrooms": 4,
        "bathrooms": 3,
        "square_feet": 2500,
        "price": 320000,
        "year_built": 2005
    }
]

**Let's begin by constructing the prompt. The first step is to design a layout for the data.**

In [11]:
def house_info_layout(houses):
    layout = ''

    for house in houses:
        layout += (
            f"House located at {house['address']}, {house['city']}, {house['state']} {house['zip']} with "
            f"{house['bedrooms']} bedrooms, {house['bathrooms']} bathrooms, "
            f"{house['square_feet']} sq feet area, priced at ${house['price']}, "
            f"built in {house['year_built']}.\n"
        )
    return layout

In [12]:
print(house_info_layout(house_data))

House located at 123 Maple Street, Springfield, IL 62701 with 3 bedrooms, 2 bathrooms, 1500 sq feet area, priced at $230000, built in 1998.
House located at 456 Elm Avenue, Shelbyville, TN 37160 with 4 bedrooms, 3 bathrooms, 2500 sq feet area, priced at $320000, built in 2005.



**Now create a function that generates the prompt to be passed to the Language Learning Model (LLM). The function will take a user-provided query and the available housing data as inputs to effectively address the user's query.**

In [13]:
def generate_prompt(query, house):
    house_layout = house_info_layout(house)
    PROMPT = f"""
    Use the following houses information to answer the user queries.
    {house_layout}
    Query: {query}
    """
    return PROMPT

In [14]:
print(generate_prompt("What is the most expensive house?", house = house_data))


    Use the following houses information to answer the user queries.
    House located at 123 Maple Street, Springfield, IL 62701 with 3 bedrooms, 2 bathrooms, 1500 sq feet area, priced at $230000, built in 1998.
House located at 456 Elm Avenue, Shelbyville, TN 37160 with 4 bedrooms, 3 bathrooms, 2500 sq feet area, priced at $320000, built in 2005.

    Query: What is the most expensive house?
    


**Now make the call to the model without passing the housing data and observe the output -**

In [15]:
query = "What is the most expensive house?"

query_without_house_info = generate_with_single_input(prompt = query, role = 'user')

query_without_house_info

{'content': 'The title of "most expensive house" can be subjective and may vary depending on various factors such as location, size, amenities, and market conditions. However, here are some of the most expensive houses in the world:\n\n1. **Antilia**, Mumbai, India - Estimated value: $1 billion (approximately ₹7,500 crore)\n\t* Owned by business magnate Mukesh Ambani, Antilia is a 27-story skyscraper that serves as his private residence.\n2. **The One**, Bel Air, California, USA - Estimated value: $500 million\n\t* This mega-mansion was designed by Paul McClean and features 20 bedrooms, 30 bathrooms, and over 105,000 square feet of living space.\n3. **Villa Leopolda**, Villefranche-sur-Mer, France - Sold for: €750 million (approximately $850 million)\n\t* This luxurious villa is situated on the French Riviera and boasts stunning views of the Mediterranean Sea.\n4. **The Biltmore Estate**, Asheville, North Carolina, USA - Estimated value: $400-500 million\n\t* This grand chateau-style m

In [16]:
print(query_without_house_info['content'])

The title of "most expensive house" can be subjective and may vary depending on various factors such as location, size, amenities, and market conditions. However, here are some of the most expensive houses in the world:

1. **Antilia**, Mumbai, India - Estimated value: $1 billion (approximately ₹7,500 crore)
	* Owned by business magnate Mukesh Ambani, Antilia is a 27-story skyscraper that serves as his private residence.
2. **The One**, Bel Air, California, USA - Estimated value: $500 million
	* This mega-mansion was designed by Paul McClean and features 20 bedrooms, 30 bathrooms, and over 105,000 square feet of living space.
3. **Villa Leopolda**, Villefranche-sur-Mer, France - Sold for: €750 million (approximately $850 million)
	* This luxurious villa is situated on the French Riviera and boasts stunning views of the Mediterranean Sea.
4. **The Biltmore Estate**, Asheville, North Carolina, USA - Estimated value: $400-500 million
	* This grand chateau-style mansion was built by George

**Now make the query by passing the housing data and observe the output.**

In [17]:
query = "What is the most expensive house?"

enhanced_query = generate_prompt(query, house=house_data)
query_with_house_info = generate_with_single_input(prompt = enhanced_query, role='assistant')

In [18]:
print(query_with_house_info['content'])

 - The most expensive house is the one located at 456 Elm Avenue, Shelbyville, TN 37160.


### Conclusion

Augmenting data with prompts involves enriching the input provided to a LLM by including relevant context, facts, or structured data before sending it for inference. Instead of relying solely on the LLM's pre-trained knowledge, you supply domain-specific details—such as product specs, financial figures, or a subset of a dataset—directly in the prompt.

This approach is critical in **Retrieval-Augmented Generation (RAG)** systems, where data from external sources (e.g., JSON, databases, vector stores) is retrieved and embedded into the prompt. By doing so:

* **Accuracy Improves**: The model has access to the exact, up-to-date information instead of hallucinating or giving generic responses.
* **Computation Is Faster**: Since the model does not need to reason broadly or infer missing details, the search space for generating answers is reduced, leading to quicker response times and lower token usage.

In short, prompt augmentation creates a balance between LLM general intelligence and real-time factual accuracy while improving efficiency.


### Acknowledgement

This blog draws inspiration from the RAG course by **deeplearning.ai**. However, instead of using the OpenAI API with Together.ai, as demonstrated in the course, I’ve implemented the solution using **Ollama**, an open-source, quantized model that can even run on your CPU. While responses might take a few seconds depending on your input data, this approach eliminates the need for external APIs.

You can check out the original course here: [Retrieval-Augmented Generation (RAG) on Coursera](https://www.coursera.org/learn/retrieval-augmented-generation-rag/).